In [13]:
!pip install -r requirements.txt

In [14]:
import base64
import json

from cryptography.hazmat.primitives import serialization
from cryptography.hazmat.backends import default_backend
from cryptography.hazmat.primitives.asymmetric import rsa, padding
from cryptography.hazmat.primitives import hashes
from cryptography.fernet import Fernet

### make public and private pem keypair

In [15]:
!sh keys/make.sh

.......+.+.....+............+................+..+..........+..+.+..+.......+..+....+.....+.+.....+...+.+......+......+..+...+...+.+.....+.+............+...+..+...+....+...+.....+.+.....+.+......+.........+.....................+..+.+.....+.+.....+..........+.........+..+++++++++++++++++++++++++++++++++++++++++++++*..+....+...+..+.+++++++++++++++++++++++++++++++++++++++++++++*........+.........................+.....+.+......+...........+....+........+..........+.................+....+..............+...+.......+..+.........+................+...+..+....+..+.......+.....+...+....+......+..+................+.....+...................+..+.+........+....+..+.+..+.......+........+...+............................+.................+....+...+.....+.......+......................................+.+..+....+..............+....+..+.+..............+...+.+......+...+.....+......+.+.....+.+............+........+....+.........+............+..+..........+..+.+..+.............+..+...+...+.......+.....+.......

### read in pub and private key

In [16]:
with open("keys/private_key.pem", "rb") as f:
    private_key = serialization.load_pem_private_key(
        f.read(), password=None, backend=default_backend()
    )

with open("keys/public_key.pem", "rb") as f:
    public_key = serialization.load_pem_public_key(f.read(), backend=default_backend())

### 1. create symetric AES key and encrypt with RSA public key

In [17]:
# Generate a new AES key for this encryption session
aes_key = Fernet.generate_key()
cipher = Fernet(aes_key)

In [18]:
# Encrypt the AES key with the RSA public key
encrypted_aes_key = public_key.encrypt(
    aes_key,
    padding.OAEP(
        mgf=padding.MGF1(algorithm=hashes.SHA256()),
        algorithm=hashes.SHA256(),
        label=None,
    ),
)

### 2. convert to base64

In [19]:
base64_encoded_key = base64.b64encode(encrypted_aes_key).decode("utf-8")

In [20]:
base64_encoded_key

'ewx1QWhonBS+YhMgelgD17eq1SxsrO8JZFuLF0cGZi7yJbrW1iqe4aWo/nwBdPtoDP4snQC2lZiE6E3fn9r5s6SFztOy4TO1CSHBiOZ1Fi57rYVQsuONQejY7Da9027UmKFcseo+GusmpsvZ+LRTqjPeiB8MVATb/E01a1jH7qRd2MAU9lKY1EvfkyJ4j1+niYUAamF+6XZh5CK2uUPC3psJ5d8Tf0udoNevsh6fxqsLXfbt40arCXhDC4nBRxa2oXRERw5/8UW/d4URU9SZXbWLZUMyG7Aq79Qve58ibTwU7i3DF45LBn2qOSuIDlEGRK55tR8eU0jmEglzHKx3JiVbZIoNsEvKdbvF52nlrjGqpEaoqrquHFATb/9AEH+odnVkcEvk1HjGZXfSI5X3TSejncBXLmUdMe/gRJ7mfSX4p6GjS2PhHlKgv36GWngEZqJ32Ye9OlJI4QVhUP/NYIexZkOLwNrA/yXtOi0EHApG1QTlq9FtsGJlvABX3h7eEpIlKHDvqQcubA+bh8vEMEgxzoEU3AapVyVt+4NcjV8SdZLFQ0Ize4fwi4NpBmltgUxTHvH+ckK91P14qZZRbUvv2aUsrhLmfW+GcGSdtpMktQDhC2RLGttntjm6R9qq38R1cDXJ9ORUN66tHOClUZMqCvBrFZzoi8VHYDVio9w='

### 3. encrypt test email and write to response.json

In [21]:
# Define encryption function for data
def encrypt_value(value):
    if isinstance(value, str):
        return cipher.encrypt(value.encode()).decode()
    return None

In [22]:
email = "loremipsum@email.com"
email_encrypted = encrypt_value(email)

In [23]:
output = {
    "emailEncrypted": email_encrypted,
    "base64EncodedKey": [base64_encoded_key],
}

with open("response/response.json", "w") as f:
    json.dump(output, f)